In [2]:
#Imports
#Imports
import numpy as np
import pandas as pd
import seaborn as sns
import ast, json
import math
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
from statsmodels.stats import weightstats as stests
import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from yellowbrick.regressor import ResidualsPlot
from math import sqrt
from sklearn.model_selection import KFold, cross_val_score
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn import svm
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn import datasets
 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble.forest import RandomForestRegressor

import qgrid
from sklearn.preprocessing import scale, normalize
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
%matplotlib inline

In [3]:
#Useful methods
def numerify_df(data):
    for c in data.columns:
        try:
            data[c] = data[c].astype(float)
        except:
            data = data.drop(columns=c)
    return data

def get_elements(df):
    new_cols = []
    for c in df.columns:
        if len(c)<3:
            if c is not 'A5' and c is not 'Rm':
                new_cols.append(c)
            
    return df[new_cols]


In [4]:
def get_merged_data():
    #Load initial data
    new_data = pd.read_csv('../data/final_data/02_23_newdata.csv').drop(columns='Unnamed: 0')
    new_data['Category'] = 'Stick'
    print("NEW DATA" + str(new_data.shape))
    linc_data = pd.read_csv('../data/final_data/02_23_lincdata.csv').drop(columns='Unnamed: 0')
    print("LINC DATA" + str(linc_data.shape))
    weld_data = pd.read_csv('../data/final_data/02_23_welddata.csv').drop(columns='Unnamed: 0')
    print("CEWELD DATA" + str(weld_data.shape))
    cit_data = pd.read_csv('../data/cit_data/200124_cit_data.csv').drop(columns='Unnamed: 0')
    print("CIT DATA" + str(cit_data.shape))
    
    take_smaller = lambda s1, s2: s1 if s1.sum() < s2.sum() else s2
    new_data['UTS'] = new_data['UTS'].combine(new_data['UTS(Kg/mm2)'], take_smaller)
    new_data = new_data.drop(columns={'UTS(Kg/mm2)','Elong %'})
    
    
    #line up datasets and merge them
    new_data.columns = ['Diffusible Hydrogen', 'Moisture', 'Yield strength', 'Tensile', 'Hardness',
           'Ferrite (Fn)', 'Hardness_Scale', 'V', 'C', 'Cr', 'Mn', 'Mo', 'Ni',
           'P ', 'S ', 'Si', 'Cb', 'Cu', 'Category', 'Elongation']

    linc_data.columns = ['As', 'C', 'Charpy', 'Cr', 'Cu', 'Diffusible Hydrogen', 'Elongation',
           'Lateral', 'Mn', 'Mn + Ni + Cr + Mo + V', 'Mo', 'N', 'Nb', 'Ni',
           'Ni+Mn', 'P', 'P S', 'S', 'S Ni', 'SN', 'Si', 'Si P', 'Sn', 'Tensile',
           'V', 'Yield strength', 'aws', 'conformances', 'ind', 'key features',
           'requirements', 'typic_results', 'typical applications', 'Category',
           'name', 'url']

    new_data = new_data.append(linc_data, ignore_index=True, sort=False)
    new_data = new_data.append(weld_data, ignore_index=True, sort=False)
    new_data = new_data.append(cit_data, ignore_index=True, sort=False)
    
    
    #Merging Stick Columns
    sticks = ['7018.0', '  6013 (REST)', '  LOTHERME', '  LOW ALLOY(SPL)', '  ST. STEELS', 'stainless-high-alloy', 'SMAW Stick Electrodes', 'stick-electrodes', '  6013 (NORMA-V)']

    for s in sticks:
        new_data['Category'] = new_data['Category'].replace(s,'Stick')
    
    #Identifying Stick and non-Stick
    uns = new_data['Category'].unique()[1:]
    for u in uns:
        new_data['Category'] = new_data['Category'].replace(u,"Others")
        new_data['Category'].unique()
    uns = new_data['Category'].unique()

    i = 0
    for u in uns:
        new_data['Category'] = new_data['Category'].replace(u,1-i)
        i = i+1
    
    #1 means stick, 0 means other
    new_data = new_data.drop(columns={'ind',
 'Reduction of area',
 'Ultimate tensile strength',
 'Ferrite with carbide aggregate',
 'Manual metal arc_Heat input',
 'Manual metal arc_Interpass temperature',
 'Submerged arc_Heat input',
 'Submerged arc_Interpass temperature',
 'extr'})
    new_data = new_data.dropna(axis=1, thresh=100)
    
    
    return new_data

#If true it will only return stick dataset
def get_ml_data(stick):
    #Get merged dataset
    df = get_merged_data()
    
    
    #Get training dataset
    df = numerify_df(df).astype(float)
    columns = list(df.columns)
    df = df.to_numpy()
    
    #Getting rid of non-sticks
    df[df==0] = np.nan
    if stick:
        df = df[pd.notnull(df[:,columns.index("Category")])]
    
    tmp = pd.DataFrame(df, columns = columns)
    elems = get_elements(tmp).drop(columns={'A5', 'Rm'})
    elem_cols = elems.columns
    elems = numerify_df(elems).astype(float).to_numpy()
    elems[elems==0] = np.nan
    
    
        
    return df, elems, elem_cols, columns

#Getting all of the columns of parameters to predict
def get_pred_cols():
    pred_cs = ['Rm', 'A5']
    for c in columns:
        if len(c)>2:
            pred_cs.append(c)
    return pred_cs

In [5]:
#Methods for pre-examining data

def get_model_err(df,prop, model):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    #tmp[tmp==0] = np.nan
    #tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    print(y)
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    lin_reg = model
    lin_reg.fit(X_train, y_train)
    pred = lin_reg.predict(X_test)
    err = sqrt(mean_squared_error(y_test, pred))
    return err

def get_model_residual(df,prop, model):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    #tmp[tmp==0] = np.nan
    #tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    #Trying data transformation
    X_train = X_train+0.0000000000000001
    X_test = X_test+0.0000000000000001
    X_train = np.power(X_train,-.2)
    X_test = np.power(X_test,-.2)
    lin_reg = ResidualsPlot(model)
    lin_reg.fit(X_train, y_train)
    lin_reg.score(X_test, y_test)
    return lin_reg

def try_model(model):
    a = []
    for c in get_pred_cols():
        err = get_model_err(df, c,model)
        a.append(err)
    i = 0
    zers = []
    for c in a:
        if c==0:
            zers.append(get_pred_cols()[i])
        print(get_pred_cols()[i] +" "+str(c))
        i+=1
    return zers
        
def get_res_plots(model):
    a = []
    for c in get_pred_cols():
        err = get_model_residual(df, c,model)
        print(columns[i])
        err.show()
        a.append(err)


In [6]:
df, elems, elem_cols, columns = get_ml_data(True)

NEW DATA(1509, 22)
LINC DATA(944, 36)
CEWELD DATA(678, 51)
CIT DATA(1650, 75)


In [7]:
pandadata = pd.DataFrame(df, columns= columns)
print(get_pred_cols())
qgrid.show_grid(pandadata)

['Rm', 'A5', 'Diffusible Hydrogen', 'Moisture', 'Yield strength', 'Tensile', 'Ferrite (Fn)', 'Category', 'Elongation', 'Charpy', 'DIN_W', 'HT_Temp', 'IE-20', 'IE-40', 'IE-60', 'Redry_Time', 'Rp0']


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
try_model(RandomForestRegressor())

[  0.   0.   0. ... 430. 500. 280.]
[ 0.  0.  0. ...  0. 10.  0.]
[0. 5. 5. ... 0. 0. 0.]
[0.  0.3 0.3 ... 0.  0.  0. ]
[  0. 200. 200. ...   0.   0.   0.]
[  0. 550. 550. ...   0.   0.   0.]
[0. 0. 0. ... 0. 0. 0.]
[1. 1. 1. ... 1. 1. 1.]
[ 0. 11. 11. ...  0.  0.  0.]
[0. 0. 0. ... 0. 0. 0.]
[0.     0.     0.     ... 0.     0.     2.1025]


/home/prasann/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 1. 1. 2.]
[  0.   0.   0. ...   0. 350. 120.]
Rm 78.33190310752646
A5 3.860297078497216
Diffusible Hydrogen 0.5262908604177822
Moisture 0.016193296028648656
Yield strength 82.05978605977151
Tensile 100.8842089545413
Ferrite (Fn) 0.7932983361704499
Category 0.0
Elongation 4.3379933782990605
Charpy 21.119753363538404
DIN_W 0.13722751221086144
HT_Temp 27.01367498811734
IE-20 23.47501284795281
IE-40 18.151114416075245
IE-60 6.227109973687223
Redry_Time 0.1526048846801058
Rp0 8490.092398910017


['Category']

In [9]:
#Very computationally expensive
#t = try_model(svm.SVR(kernel = 'linear', C=1000))

In [10]:
try_model(XGBRegressor())

[  0.   0.   0. ... 430. 500. 280.]
[19:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[ 0.  0.  0. ...  0. 10.  0.]
[19:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0. 5. 5. ... 0. 0. 0.]
[19:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.  0.3 0.3 ... 0.  0.  0. ]
[19:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[  0. 200. 200. ...   0.   0.   0.]
[19:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[  0. 550. 550. ...   0.   0.   0.]
[19:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0. 0. 0. ... 0. 0. 0.]
[19:24:54] WARNING: /w

[]

In [48]:
#trying out Gaussian Processes
import sklearn.gaussian_process as gp
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1, normalize_y=True)
get_model_err(df, "Charpy",model)

KeyboardInterrupt: 

In [39]:
#Hyperparameter optimization code 2

models = {
    'logistic_regression': LogisticRegression,
    'rf': RandomForestRegressor,
    'knn': KNeighborsRegressor, 'svr': SVR,
    'xgb': XGBRegressor
}


def search_space(model):

    model = model.lower()
    space = {}

    if model == 'knn':
        space = {'n_neighbors': hp.choice('n_neighbors', range(1, 100)),


                 }

    elif model == 'xgb':
        space = {'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
    'silent':True,
                 }

    elif model == 'svr':
        space = {
            'C': hp.uniform('C', 0, 20),
            'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
            'gamma': hp.uniform('gamma', 0, 20),
        }

    
    elif model == 'rf':
        space = {'max_depth': hp.choice('max_depth', range(1, 20)),
                 'max_features': hp.choice('max_features', range(1, 10)),
                 'n_estimators': hp.choice('n_estimators', range(10, 50)),
                 'criterion': hp.choice('criterion', ['mse', 'mae']),
                 }
        
    elif model == 'cat':
        space = {'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'max_depth':         hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
    'colsample_bylevel': hp.choice('colsample_bylevel', np.arange(0.3, 0.8, 0.1)),
    'n_estimators':      100,
    'eval_metric':       'RMSE',
    'silent':True
                 }
    elif model == 'lgb':
        space = {'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
                 'silent':True
                 }
        
    elif model == 'logistic_regression':
        space = {
            'warm_start': hp.choice('warm_start', [True, False]),
            'fit_intercept': hp.choice('fit_intercept', [True, False]),
            'tol': hp.uniform('tol', 0.00001, 0.0001),
            'C': hp.uniform('C', 0.05, 3),
            'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
            'max_iter': hp.choice('max_iter', range(100, 1000)),
            'multi_class': 'auto',
            'class_weight': 'balanced'
        }
    return space

In [40]:
from hyperopt import space_eval
kf = KFold(n_splits=5, random_state=42)
curmodel = XGBRegressor
dataset = 1
curvar = ""
def get_data(prop):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(len(df),1)], axis=1)
    tmp = np.nan_to_num(tmp)
    
    #tmp[tmp==0] = np.nan
    #tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.333, random_state=52, shuffle=True)
    return x_val, x_test, x_train, y_val, y_test, y_train

def run_opt(prop, space):
    global curvar
    global curmodel
    print(prop)
    curmodel = models[space]
    curvar = prop
    trials = Trials()
    
    #TODO change max_evals to 100
    best = fmin(fn=objective,
                space=search_space(space),
                algo=tpe.suggest,
                max_evals=1,
                trials=trials)
    
    return trials.best_trial['result']['loss'], trials.best_trial['result']['sco'], best

def get_opt_model(prop, space):
    global curvar
    global curmodel
    #TODO Change max evals to 100
    curmodel = models[space]
    curvar = prop
    trials = Trials()
    best = fmin(fn=objective,
                space=search_space(space),
                algo=tpe.suggest,
                max_evals=1,
                trials=trials,
               return_argmin=False)
    
    model = get_model(best)
    
    return model, trials.best_trial['result']['loss'], trials.best_trial['result']['sco'], best


def get_model(space):
    global curvar
    global curmodel
    #print(space)
    if(curmodel==XGBRegressor):
        clf = curmodel(
                               **space, objective = 'reg:squarederror'
                                )
    else:
        clf = curmodel(
                               **space
                                )

    
    x_val, x_test, x_train, y_val, y_test, y_train = get_data(curvar)
    
    eval_set  = [( x_train, y_train), ( x_val, y_val)]

    if(curmodel==XGBRegressor):
        clf.fit(x_train, y_train,
                eval_set=eval_set, eval_metric="rmse",
                early_stopping_rounds=10,verbose=False,)
    else:
        clf.fit(x_train, y_train)

    pred = clf.predict(x_test)
    mse_scr = -cross_val_score(clf, x_train, y_train, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1).mean()
    #print ("SCORE:", np.sqrt(mean_squared_error(y_test, pred)))
    #change the metric if you like|
    pred = clf.predict(x_test)
    score = np.sqrt(mean_squared_error(y_test, pred))
    return clf

def objective(space):
    global curvar
    global curmodel
    #print(space)
    if(curmodel==XGBRegressor):
        clf = curmodel(
                               **space, objective = 'reg:squarederror'
                                )
    else:
        clf = curmodel(
                               **space
                                )

    x_val, x_test, x_train, y_val, y_test, y_train = get_data(curvar)
    
    eval_set  = [( x_train, y_train), ( x_val, y_val)]

    print(clf)
    if(curmodel==XGBRegressor):
        clf.fit(x_train, y_train,
                eval_set=eval_set, eval_metric="rmse",
                early_stopping_rounds=10,verbose=False,)
    else:
        clf.fit(x_train, y_train)

    pred = clf.predict(x_test)
    mse_scr = -cross_val_score(clf, x_train, y_train, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1).mean()
    #print ("SCORE:", np.sqrt(mean_squared_error(y_test, pred)))
    #change the metric if you like
    pred = clf.predict(x_test)
    score = np.sqrt(mean_squared_error(y_test, pred))
    return {'loss':np.sqrt(mse_scr), 'status': STATUS_OK, 'sco':score }

In [41]:
#Code for getting benchmarks using hyperopt code 2
def get_benchmarks():
    d = pd.DataFrame()
    a = {}
    a['data'] = dataset
    mods = []
    for m in models.keys():
        print(m)
        a['model'] = m
        for c in get_pred_cols():
            if c is not 'Category':
                print(c)
                a['column'] = c
                mod, v, t, p = get_opt_model(c, m)
                a['validation'], a['test'], a['params'] = run_opt(c, m)
                d = d.append(a, ignore_index=True, sort=False)
                mods.append(mod)
    return d, mods

def get_models(prop):
    d = []
    for c in get_pred_cols():
        if c is not 'Category':
            d.append(get_opt_model(c, prop))
    return d

In [ ]:
get_models('svr')

SVR(C=19.25734549517949, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=16.349979347749525, kernel='sigmoid', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
100%|██████████| 1/1 [00:01<00:00,  1.12s/trial, best loss: 149.06805397687774]
SVR(C=12.801313708145468, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=18.497038972106893, kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

In [166]:
d = get_benchmarks()
d.to_csv('../benchmarks/benchmark_2.csv')

logistic_regression
Rm
100%|██████████| 10/10 [00:02<00:00,  4.72trial/s, best loss: 1.33514404296875e-05]
A5
100%|██████████| 10/10 [00:03<00:00,  3.30trial/s, best loss: 1.33514404296875e-05]
Diffusible Hydrogen
100%|██████████| 10/10 [00:02<00:00,  3.43trial/s, best loss: 1.33514404296875e-05]
Moisture
100%|██████████| 10/10 [00:02<00:00,  4.98trial/s, best loss: 1.33514404296875e-05]
Yield strength
100%|██████████| 10/10 [00:02<00:00,  3.39trial/s, best loss: 1.33514404296875e-05]
Tensile
100%|██████████| 10/10 [00:02<00:00,  3.45trial/s, best loss: 1.33514404296875e-05]
Hardness
100%|██████████| 10/10 [00:02<00:00,  4.77trial/s, best loss: 1.33514404296875e-05]
Ferrite (Fn)
100%|██████████| 10/10 [00:02<00:00,  3.46trial/s, best loss: 1.33514404296875e-05]
Category
100%|██████████| 10/10 [00:02<00:00,  3.46trial/s, best loss: 1.33514404296875e-05]
Elongation
100%|██████████| 10/10 [00:01<00:00,  5.05trial/s, best loss: 1.33514404296875e-05]
Charpy
100%|██████████| 10/10 [00:02<00: